In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:

* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

In [1]:
import graphlab

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int)

This non-commercial license of GraphLab Create for academic use is assigned to mikael.baymani@gmail.com and will expire on May 13, 2020.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1565601081.log


In [3]:
import numpy as np
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # add a constant column to an SFrame
    # prepend variable 'constant' to the features list
    features = ['constant'] + features
    # select the columns of data_SFrame given by the ‘features’ list into the SFrame ‘features_sframe’
    features_sframe = graphlab.SFrame()
    for feature in features:
        features_sframe[feature] = data_sframe[feature]

    # this will convert the features_sframe into a numpy matrix with GraphLab Create >= 1.7!!
    features_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the target to the variable ‘output_sarray’
    output_sarray = data_sframe[output]

    # this will convert the SArray into a numpy array:
    output_array = output_sarray.to_numpy() # GraphLab Create>= 1.7!!
    return(features_matrix, output_array)

In [4]:
def predict_output(feature_matrix, weights):
    # ŷ_i = h^T (x_i) * w
    predictions = np.dot(feature_matrix, weights)
    return predictions

In [7]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    X = feature_matrix / norms
    return  (X,norms)

In [9]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


# Implementing Coordinate Descent with normalized features

In [10]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [11]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [12]:
weights = np.array([1., 4., 1.])

In [14]:
prediction = predict_output(simple_feature_matrix, weights)

In [15]:
# Compute the values of ro[i] for each feature in this simple model

# ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]

In [24]:
print "ro[0] = " +str(np.dot(simple_feature_matrix[:,0],(output - prediction + weights[0]*simple_feature_matrix[:,0])))
print "ro[1] = " +str(np.dot(simple_feature_matrix[:,1],(output - prediction + weights[1]*simple_feature_matrix[:,1])))
print "ro[2] = " +str(np.dot(simple_feature_matrix[:,2],(output - prediction + weights[2]*simple_feature_matrix[:,2])))

ro[0] = 79400300.03492917
ro[1] = 87939470.77299108
ro[2] = 80966698.67596565


In [25]:
# QUIZ QUESTION

# Recall that, whenever ro[i] falls between -l1_penalty/2 and l1_penalty/2, the
# corresponding weight w[i] is sent to zero. Now suppose we were to take one step
# of coordinate descent on either feature 1 or feature 2. What range of values of
# l1_penalty would not set w[1] zero, but would set w[2] to zero, if we were to
# take a step in that coordinate?

# -lambda/2 <= r[2] <= lambda/2
# -lambda/2 <= 80,966,699 <= lambda/2   ==>   -87,939,470 <= 80,966,699 <= 87,939,470
# lambda/2 = 87,939,470                 ==>   1.6e8 <= lambda <= 2 * 87,939,470 = 175,878,940


In [26]:
# QUIZ QUESTION

# What range of values of l1_penalty would set both w[1] and w[2] to zero, if we were
# to take a step in that coordinate?

# lambda > 175,878,940

In [30]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.dot(feature_matrix[:,i],(output - prediction + weights[i]*feature_matrix[:,i]))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

In [31]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.4255588466910251


In [62]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    
    converged = None
    weights = np.array(initial_weights);
    
    while not converged:
        
        converged = True

        for i in range(len(weights)):
            old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
            # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
            #     and old values for weight[i], ..., weight[d-1]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)

            # use old_weights_i to compute change in coordinate
            change_in_coordinate = abs(old_weights_i - weights[i])
            
            if change_in_coordinate > tolerance:
                    converged = False
            
    return weights

In [63]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [64]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

In [65]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [66]:
print weights

[21624998.36636291 63157246.78545423        0.        ]


In [70]:
# QUIZ QUESTIONS

# 1) What is the RSS of the learned model on the normalized dataset?
#    (Hint: use the normalized feature matrix when you make predictions.)
pred = predict_output(normalized_simple_feature_matrix, weights)
RSS = lambda output, predictions : sum((output - predictions)**2)
print RSS(output, pred)
# 2) Which features had weight zero at convergence?
#    Answer: bedrooms

1630492481484488.0


## Evaluating LASSO fit with more feature

In [71]:
train_data,test_data = sales.random_split(.8,seed=0)
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [73]:
(all_feature_matrix, output) = get_numpy_data(train_data, all_features, 'price')
(normalized_all_feature_matrix, norms) = normalize_features(all_feature_matrix)

In [75]:
initial_weights = np.zeros(1+len(all_features))
weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                               initial_weights, 1e7, 1)

In [76]:
# QUIZ QUESTION

# What features had non-zero weight in this case?

In [78]:
print [all_features[i-1] for i in range(len(weights1e7)) if i != 0 and weights1e7[i] != 0]

['sqft_living', 'waterfront', 'view']


In [79]:
print weights1e7

[24429600.60933314        0.                0.         48389174.35227978
        0.                0.          3317511.16271981  7329961.9848964
        0.                0.                0.                0.
        0.                0.        ]


In [80]:
# Next, learn the weights with l1_penalty=1e8, on the training data.
# Initialize weights to all zeros, and set the tolerance=1. Call resulting
# weights weights1e8, you will need them later.

In [81]:
initial_weights = np.zeros(1+len(all_features))
weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                               initial_weights, 1e8, 1)

In [82]:
# QUIZ QUESTION

#What features had non-zero weight in this case?

In [83]:
print [all_features[i-1] for i in range(len(weights1e8)) if i != 0 and weights1e8[i] != 0]

[]


In [84]:
weights1e8

array([71114625.75280938,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [85]:
# Finally, learn the weights with l1_penalty=1e4, on the training data. Initialize
# weights to all zeros, and set the tolerance=5e5. Call resulting weights weights1e4,
# you will need them later. (This case will take quite a bit longer to converge than the others above.)
initial_weights = np.zeros(1+len(all_features))
weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                               initial_weights, 1e4, 5e5)

In [86]:
# QUIZ QUESTION

# What features had non-zero weight in this case?

In [87]:
print [all_features[i-1] for i in range(len(weights1e4)) if i != 0 and weights1e4[i] != 0]

['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


In [90]:
weights1e4_normalized = weights1e4 / norms
weights1e7_normalized = weights1e7 / norms
weights1e8_normalized = weights1e8 / norms
# To check your results, if you call normalized_weights1e7 the normalized version of weights1e7, then:

# print normalized_weights1e7[3]
# should return 161.31745624837794.
print weights1e7_normalized[3]

161.3174562483786


## Evaluating each of the learned models on the test data

In [91]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

In [92]:
pred1e4 = predict_output(test_feature_matrix, weights1e4_normalized)
pred1e7 = predict_output(test_feature_matrix, weights1e7_normalized)
pred1e8 = predict_output(test_feature_matrix, weights1e8_normalized)

In [95]:
# QUIZ QUESTION

# Which model performed best on the test data?
print RSS(test_output, pred1e4)
print RSS(test_output, pred1e7)
print RSS(test_output, pred1e8)

227781004760225.0
275962079909184.7
537166150034084.06
